# An Introduction to Subensembles in Probly

The goal of this notebook is to showcase the subensemble transformation and how to use it.

## Different Examples from Different Libraries

The supported libraries are Flax nnx and Torch.<br>

Define simple models for both libaries to showcase the subensemble transformation.






In [ ]:
import torch
from torch import nn

class TorchModel(nn.Module):
    def __init__(self) -> None:
        "Initialize the neural network model."
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)  
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(16, 8)  
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(8, 4)
        self.relu4 = nn.ReLU()
        self.fc5 = nn.Linear(4, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        "Forward pass of the neural network."
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        return x

pytorch_model = TorchModel()
print(pytorch_model)

In [ ]:
import flax
import jax
from flax import nnx

class FlaxModel(nnx.Module, pytree=False):
    def __init__(self, *, rngs: nnx.Rngs) -> None:
        "Initialize the neural network model."
        self.layers = [
            nnx.Linear(784, 64, rngs=rngs),
            nnx.Linear(64, 32, rngs=rngs),
            nnx.Linear(32, 16, rngs=rngs),
            nnx.Linear(16, 8, rngs=rngs),
            nnx.Linear(8, 10, rngs=rngs),
        ]

    def __call__(self, x : jax.Array) -> jax.Array:
        "Forward pass of the neural network."
        x = x.reshape((x.shape[0], -1))
        x = nnx.relu(self.layers[0](x))
        x = nnx.relu(self.layers[1](x))
        x = nnx.relu(self.layers[2](x))
        x = nnx.relu(self.layers[3](x))
        return self.layers[4](x)

rng = nnx.Rngs(0)
flax_model = FlaxModel(rngs=rng)
nnx.display(flax_model)

Now we have to simple models from two different libraries and can use them to transform them into an subensemble.

## Down to Business

We can do different types of subensembles where we can choose the number of layers for heads or pass a model directly to be the head.<br>
In the following we show the different ways to use the subensemble transformation.

### 1. Default Example

We give the transformation a model and the number of heads but do not pass anymore arguments.

In [ ]:
from probly.transformation.subensemble import subensemble
num_heads = 3

subensemble_flax = subensemble(flax_model, num_heads=num_heads)
nnx.display(subensemble_flax)
subensemble_pytorch = subensemble(pytorch_model, num_heads=num_heads)
print(subensemble_pytorch)

Note, that the default layer used for the heads is the last layer as shown in the example above.

### 2. Head_Layer>1 Example

We give the transformation a model, mumber of heads and the number of layers for the heads.

In [ ]:
from probly.transformation.subensemble import subensemble
num_heads = 3
head_layer = 2

subensemble_flax = subensemble(flax_model, num_heads=num_heads, head_layer=head_layer)
nnx.display(subensemble_flax)
subensemble_pytorch = subensemble(pytorch_model, num_heads=num_heads, head_layer=head_layer)
print(subensemble_pytorch)

Note, that the last two layers are used for the head if explicitly passed with the argument head_layer.

### 3. Full-model-as-head Example

We give the transformation a model, number of heads and a model to be used as head.

#### 3.1 TorchModel to pass as a head.

In [ ]:
import torch
from torch import nn

class TorchHeadModel(nn.Module):
    def __init__(self) -> None:
        "Initialize the neural network model."
        super().__init__()
        self.fc1 = nn.Linear(10, 8)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(8, 8)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(8, 10)

    def forward(self, x : torch.Tensor) -> torch.Tensor:
        "Forward pass of the neural network."
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Example
torch_head_model = TorchHeadModel()
print(torch_head_model)


#### 3.2 FlaxModel to pass as a head.

In [ ]:
import flax
import jax
from flax import nnx

class FlaxHeadModel(nnx.Module, pytree=False):
    def __init__(self, *, rngs: nnx.Rngs) -> None:
        "Initialize the neural network model."
        self.layers = [
            nnx.Linear(10, 8, rngs=rngs),
            nnx.Linear(8, 8, rngs=rngs),
            nnx.Linear(8, 10, rngs=rngs),
        ]

    def __call__(self, x : jax.Array) -> jax.Array:
        "Forward pass of the neural network."
        x = x.reshape((x.shape[0], -1))
        x = nnx.relu(self.layers[0](x))
        x = nnx.relu(self.layers[1](x))
        return self.layers[2](x)

rng = nnx.Rngs(0)
flax_head_model = FlaxHeadModel(rngs=rng)
nnx.display(flax_head_model)

#### 3.3 Transformation

In [ ]:
from probly.transformation.subensemble import subensemble
num_heads = 3
head_flax = flax_head_model
head_torch = torch_head_model 

subensemble_flax = subensemble(flax_model, num_heads=num_heads, head=head_flax)
nnx.display(subensemble_flax)
subensemble_pytorch = subensemble(pytorch_model, num_heads=num_heads, head=head_torch)
print(subensemble_pytorch)

Note, that the passed model for the head is in every head but be careful with the right dimensions.

### 4. Params for the heads

The subensemble transformation supports resetting the parameters in the different heads.<br>
To enable resetting one has to pass it as an argument.

#### 4.1 Resetting parameters


In [ ]:
from probly.transformation.subensemble import subensemble
import numpy as np
num_heads = 3
reset_params = True

subensemble_flax = subensemble(flax_model, num_heads=num_heads, reset_params=reset_params)

heads = subensemble_flax[1]
for i in range(num_heads):
    params_heads = jax.tree_util.tree_leaves(heads[i])
    weights = np.array(params_heads[1])  
    weights_rounded = np.round(weights, 2)  
    
    print(f"\nHead {i} weights:")
    for row in weights_rounded:
        print(row)

subensemble_pytorch = subensemble(pytorch_model, num_heads=num_heads, reset_params=reset_params)

backbone, heads = subensemble_pytorch

for i, head in enumerate(heads):
    print(f"\nHead {i} weights:")
    for name, param in head.named_parameters():
        if 'weight' in name:
            w = param.data.cpu().numpy()  
            print(np.round(w, 2))



Note, that the heads have different values as weigths.

#### 4.2 Not-Resetting Parameters



In [ ]:
from probly.transformation.subensemble import subensemble
import numpy as np
num_heads = 3
reset_params = False

subensemble_flax = subensemble(flax_model, num_heads=num_heads, reset_params=reset_params)

heads = subensemble_flax[1]
for i in range(num_heads):
    params_heads = jax.tree_util.tree_leaves(heads[i])
    weights = np.array(params_heads[1])  
    weights_rounded = np.round(weights, 2)  
    
    print(f"\nHead {i} weights:")
    for row in weights_rounded:
        print(row)

subensemble_pytorch = subensemble(pytorch_model, num_heads=num_heads, reset_params=reset_params)

backbone, heads = subensemble_pytorch

for i, head in enumerate(heads):
    print(f"\nHead {i} weights:")
    for name, param in head.named_parameters():
        if 'weight' in name:
            w = param.data.cpu().numpy()  
            print(np.round(w, 2))



Note, that the heads have the same values as weigths.